# 1. PDF

In [ ]:
import pandas as pd
data = {
    "Name": ["John", "Alice", "Bob"],
    "Age": [28, 24, 32],
    "City": ["Bangkok", "Tokyo", "Seoul"]
}
df = pd.DataFrame(data)

In [ ]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', size=12)

pdf.set_font('Arial', size=16, style='B')
pdf.cell(200,10, txt="My First PDF in Python", ln=True, align="C")

pdf.set_font('Arial', size=12)
pdf.multi_cell(0,10, txt="This is created using Python")

pdf.image(r'01_DataScience\ExportFile\picture5.png', x=10, y=None, w=190)

pdf.set_font('Arial', size=12)
pdf.multi_cell(0,10, txt="Plot graph success!", align='C')


# วาด Header ตาราง
# -----------------------
col_width = pdf.w / (len(df.columns) + 1)   # คำนวณความกว้างอัตโนมัติ
row_height = 8

for col in df.columns:
    pdf.cell(col_width, row_height, col, border=1)
pdf.ln(row_height)

# -----------------------
# วาดข้อมูลบรรทัดใน DataFrame
# -----------------------
for index, row in df.iterrows():
    for col in df.columns:
        pdf.cell(col_width, row_height, str(row[col]), border=1)
    pdf.ln(row_height)


pdf.output(r'01_DataScience\ExportFile\clcoding.pdf')
print('PDF created successfully')


PDF created successfully


# 2. Path

In [ ]:
from pathlib import Path
import pandas as pd
from datetime import datetime

# กำหนดโฟลเดอร์หลัก
base_path = Path(r"\\10.1.1.124\Project")  # เปลี่ยนเป็น path ของคุณ
target = r"D:\My Documents\sutripat.ng\OneDrive - Thainamthip\1. Work\TDWPBI_Project.xlsx"
rows = []

for item in base_path.rglob("*"):
    if item.is_file():
        # โฟลเดอร์ relative path
        folder = str(item.parent.relative_to(base_path))
        if folder == ".":
            folder = ""
            top_folder = ""
        else:
            folder = folder.replace("/", "\\") + "\\"
            # top folder คือโฟลเดอร์แรก
            top_folder = folder.split("\\")[0]

        rows.append({
            "No": len(rows) + 1,
            "Path" : str(item),
            "Project": top_folder,
            "FolderPath" : folder,
            "File": item.name,
            "ExecuteDate": datetime.now()
        })

df = pd.DataFrame(rows)

# แสดงผลโดยไม่แสดง index
display(df)

df.to_excel(target, index=False)

,No,Path,TopFolder,Folder,File,ExecuteDate
0,1,\\10.1.1.124\Project\How to install AccessData...,,,How to install AccessDatabaseEngine.txt,2025-11-18 08:47:14.004240
1,2,\\10.1.1.124\Project\Thumbs.db,,,Thumbs.db,2025-11-18 08:47:14.099493
2,3,\\10.1.1.124\Project\# BACKUP\Fix_Code_2019031...,# BACKUP,# BACKUP\,Fix_Code_20190318.zip,2025-11-18 08:47:14.461415
3,4,\\10.1.1.124\Project\# BACKUP\Project-20190920...,# BACKUP,# BACKUP\,Project-20190920.zip,2025-11-18 08:47:14.479235
4,5,\\10.1.1.124\Project\# BACKUP\Project.zip,# BACKUP,# BACKUP\,Project.zip,2025-11-18 08:47:14.488870
...,...,...,...,...,...,...
14960,14961,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-HRC.png,2025-11-18 08:52:21.507585
14961,14962,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-MK.png,2025-11-18 08:52:21.520584
14962,14963,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-SPHP.png,2025-11-18 08:52:21.533656
14963,14964,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-TT.png,2025-11-18 08:52:21.546970


In [5]:
from datetime import datetime

print(datetime.now())

2025-11-18 08:43:36.522342


# 3. Connect SQL

In [1]:
# -------------- Step 1 : Query MsSQL and DELETE Historical data 3 months --------------

from sqlalchemy import create_engine, text, DECIMAL
import urllib
import pandas as pd

# ตั้งค่า connection
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=TDWPOWERBI;"   
    "DATABASE=TNTL_PUR;"
    "UID=tntlbiadmin;"             # SQL Login user
    "PWD=admin@123;"  # รหัสผ่าน
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


# เขียน SQL Query
sql_query = """
    SELECT * 
    FROM dbo.STG_Sugar
    """

# แสดงผลลัพธ์
df = pd.read_sql(sql_query, engine)
display(df)

,YEAR,MONTH,PRICE,UPDATE DATE
0,2017,1,489.36,2025-09-04 15:16:10.377
1,2017,2,497.46,2025-09-04 15:16:10.377
2,2017,3,479.28,2025-09-04 15:16:10.377
3,2017,4,454.86,2025-09-04 15:16:10.377
4,2017,5,445.72,2025-09-04 15:16:10.377
...,...,...,...,...
101,2025,6,471.79,2025-09-29 14:41:17.847
102,2025,8,479.40,2025-11-07 10:45:13.980
103,2025,9,473.55,2025-11-07 10:45:13.980
104,2025,10,440.38,2025-11-07 10:45:13.980


# 99. TEMP

## Example 1

In [ ]:
# -------------- Step 1 : Query MsSQL and DELETE Historical data 3 months --------------

from sqlalchemy import create_engine, text, DECIMAL
import urllib
import pandas as pd

# ตั้งค่า connection
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=TDWPOWERBI;"   
    "DATABASE=TNTL_PUR;"
    "UID=tntlbiadmin;"             # SQL Login user
    "PWD=admin@123;"  # รหัสผ่าน
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


# เขียน SQL Query
sql_query = """
    SELECT * 
    FROM dbo.STG_Sugar
    """

# แสดงผลลัพธ์
df = pd.read_sql(sql_query, engine)
display(df)

In [ ]:
df = df.set_index('YEAR')
df = df.groupby('YEAR').agg({'PRICE':['min','max','mean','count']}).reset_index()
df.columns = ['_'.join(col).strip('_') for col in df.columns]
df = df.rename(columns={'PRICE_min':'MIN', 'PRICE_max':'MAX', 'PRICE_mean':'MEAN', 'PRICE_count':'COUNT'})

display(df)
x = df['YEAR']
y1 = df['MIN']
y2 = df['MAX']
y3 = df['MEAN']
y4 = df['COUNT']

import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs[0,0].plot(x,y1, 'ro--')
axs[0,0].set_title('Chart 1')

axs[0,1].plot(x,y2, 'go--')
axs[0,1].set_title('Chart 2')

axs[1,0].plot(x,y3, 'yo--')
axs[1,0].set_title('Chart 3')

axs[1,1].pie(y4, labels=x, autopct='%.2f%%', explode=[0,0,0,0,0,0,0,0,0.7])
axs[1,1].set_title('Chart 4')
